## Parse single-trial spikes/forces, save to .mat
    - Generate from single-trial spikes provided by Elom
    - No filtering of trials was performed
    - Need a time mask and a condition mask
    - also make vector of trial-average forces

In [1]:
import pickle
import os
import pdb
import numpy as np
from scipy.sparse import coo_matrix
import scipy
import matplotlib.pyplot as plt

In [174]:
# where is the data saved?
file_name = '/Users/andrew/Documents/Projects/Churchland/Pacman/data/cousteau/processed/redOnly_MC/Cousteau_single_trial_data.pkl'

# open the file
with open(file_name, 'rb') as handle:
    all_data = pickle.loads(handle.read())

# get a list of dates
dates = list(all_data.keys())

# place all the data in a dictionary keyed by the date
session_data = {date: {} for date in dates}


In [175]:
# going to export single-trial spikes and forces. Organization will be [top level directory]/recording date/condition_id

# top level directory
saveDir = '/Users/andrew/Documents/Projects/Churchland/Pacman/data/cousteau/processed/redOnly_MC/'

# cycle through recording dates
for session_date, trial_data in all_data.items():

    # put all of the spikes and force profiles into a list
    for data in trial_data:
        condition_id = data['behavior']['condition_id'][0]
        current_values = session_data[session_date].get((condition_id,), {
            'force': [],
            'spikes': []
        })
        trial_force = data['behavior']['force'][0]
        spiking_activity = data['spikes'].toarray()
        current_values['force'].append(trial_force)
        current_values['spikes'].append(spiking_activity)
        session_data[session_date][(condition_id,)] = current_values

    # concatenate force and spikes
    for key, value in session_data[session_date].items():
        stacked_force = np.vstack(value['force'])
        stacked_activity = np.stack(value['spikes'])
        value['force'] = stacked_force
        value['spikes'] = stacked_activity

    # cycle through all the conditions and export spikes and forces
    # condition ids

    # initialize a directory for this session
    session_string = session_date.strftime('%y/%m/%d')
    sessionDirName = session_string.replace('/','')
    os.mkdir(saveDir + sessionDirName)

    # directory for saving condition data
    sessDir = os.path.join(saveDir, sessionDirName)

    for id in list(session_data[session_date].keys()):

        # pull out force
        force  = session_data[session_date][id]['force']
        spikes = session_data[session_date][id]['spikes']

        # convert spikes to boolean array
        spikes = spikes > 0

        # save spikes and force
        str_id = str(id)
        str_id = str_id.replace('(','')
        str_id = str_id.replace(')','')
        str_id = str_id.replace(',','')
        scipy.io.savemat(sessDir + '/' + 'condition_' + str_id + '.mat', {'force': force, 'spikes':spikes})

